To run predictions, you first need to place the trained model file (.pt format) into the current directory

In [ ]:
import pandas as pd
import sys
sys.path.append('../../../../..chemprop')
sys.path.append('../../../../..chemprop/chemprop')
from sklearn.metrics import r2_score
from tqdm import tqdm
from typing import Callable, List
import logging

import numpy as np
import torch
from tqdm import trange
from chemprop.args import TrainArgs
from chemprop.data import get_data, MoleculeDataLoader, StandardScaler
from chemprop.models import MoleculeModel
from chemprop.utils import get_loss_func, get_metric_func, load_checkpoint

In [ ]:
df = pd.read_csv('data/separate_data/SGNN_dataset.csv')
df = df[df['dataset_type'] == 'test']
df  = df.drop(columns=['dataset_type'])
df = df.rename(columns={'molecule':'smiles', 'logp':'logP'})
df.to_csv('SGNN_dataset(test).csv', index=False)

In [6]:
def evaluate(model: MoleculeModel,
             data_loader: MoleculeDataLoader,
             num_tasks: int,
             metric_func: Callable,
             dataset_type: str,
             scaler: StandardScaler = None,
             logger: logging.Logger = None) -> List[float]:
    """
    Evaluates an ensemble of models on a dataset by making predictions and then evaluating the predictions.

    :param model: A :class:`~chemprop.models.model.MoleculeModel`.
    :param data_loader: A :class:`~chemprop.data.data.MoleculeDataLoader`.
    :param num_tasks: Number of tasks.
    :param metric_func: Metric function which takes in a list of targets and a list of predictions.
    :param dataset_type: Dataset type.
    :param scaler: A :class:`~chemprop.features.scaler.StandardScaler` object fit on the training targets.
    :param logger: A logger to record output.
    :return: A list with the score for each task based on :code:`metric_func`.
    """
    preds = predict(
        model=model,
        data_loader=data_loader,
        scaler=scaler
    )

    targets = data_loader.targets()

    results, results_r2 = evaluate_predictions(
        preds=preds,
        targets=targets,
        num_tasks=num_tasks,
        metric_func=metric_func,
        dataset_type=dataset_type,
        logger=logger
    )

    return results, results_r2

In [7]:
def predict(model: MoleculeModel,
            data_loader: MoleculeDataLoader,
            disable_progress_bar: bool = False,
            scaler: StandardScaler = None,
            args: TrainArgs = None) -> List[List[float]]:
    """
    Makes predictions on a dataset using an ensemble of models.

    :param model: A :class:`~chemprop.models.model.MoleculeModel`.
    :param data_loader: A :class:`~chemprop.data.data.MoleculeDataLoader`.
    :param disable_progress_bar: Whether to disable the progress bar.
    :param scaler: A :class:`~chemprop.features.scaler.StandardScaler` object fit on the training targets.
    :return: A list of lists of predictions. The outer list is molecules while the inner list is tasks.
    """
    model.eval()

    preds = []

    for batch in tqdm(data_loader, disable=disable_progress_bar):
        # Prepare batch
        batch: MoleculeDataset
        if args.additional_encoder:
            substructure_mol_batch = batch.batch_graph(model_type='substructures', args = args)
        mol_batch, features_batch = batch.batch_graph(args = args), batch.features()

        # Make predictions
        with torch.no_grad():
            if args.additional_encoder:        
                batch_preds = model(batch = mol_batch, substructures_batch = substructure_mol_batch, features_batch = features_batch)
            else:
                batch_preds = model(batch = mol_batch, features_batch = features_batch)

        batch_preds = batch_preds.data.cpu().numpy()

        # Inverse scale if regression
        if scaler is not None:
            batch_preds = scaler.inverse_transform(batch_preds)

        # Collect vectors
        batch_preds = batch_preds.tolist()
        preds.extend(batch_preds)

    return preds

In [8]:
class Args(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

In [9]:
Seed = 42

In [10]:
SMILES_COLUMN = 'smiles'
VALUE_COLUMN = 'logP'
PREDS_COLUMN = f'logP_pred_{Seed}'
DATASET_OUTPUT_PATH = '.'
MODEL_PATH = f'SGNN_dataset_{Seed}.pt'

In [11]:
model = load_checkpoint(MODEL_PATH, device=torch.device('cpu'))

Loading pretrained parameter "encoder.encoder.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.W_i.weight".
Loading pretrained parameter "encoder.encoder.W_h.weight".
Loading pretrained parameter "encoder.encoder.W_o.weight".
Loading pretrained parameter "encoder.encoder.W_o.bias".
Loading pretrained parameter "substructures_encoder.encoder.cached_zero_vector".
Loading pretrained parameter "substructures_encoder.encoder.W_o.weight".
Loading pretrained parameter "substructures_encoder.encoder.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".


In [12]:
def evaluate_predictions(preds: List[List[float]],
                         targets: List[List[float]],
                         num_tasks: int,
                         metric_func: Callable,
                         dataset_type: str,
                         logger: logging.Logger = None) -> List[float]:
    """
    Evaluates predictions using a metric function after filtering out invalid targets.

    :param preds: A list of lists of shape :code:`(data_size, num_tasks)` with model predictions.
    :param targets: A list of lists of shape :code:`(data_size, num_tasks)` with targets.
    :param num_tasks: Number of tasks.
    :param metric_func: Metric function which takes in a list of targets and a list of predictions.
    :param dataset_type: Dataset type.
    :param logger: A logger to record output.
    :return: A list with the score for each task based on :code:`metric_func`.
    """
    info = logger.info if logger is not None else print

    if len(preds) == 0:
        return [float('nan')] * num_tasks

    # Filter out empty targets
    # valid_preds and valid_targets have shape (num_tasks, data_size)
    valid_preds = [[] for _ in range(num_tasks)]
    valid_targets = [[] for _ in range(num_tasks)]
    for i in range(num_tasks - 1):
        for j in range(len(preds)):
            if targets[j][i] is not None:  # Skip those without targets
                valid_preds[i].append(preds[j][i])
                valid_targets[i].append(targets[j][i])

    # Compute metric
    results = []
    results_r2 = []
    
    for i in range(num_tasks):
        # Skip if all targets or preds are identical, otherwise we'll crash during classification
        if dataset_type == 'classification':
            nan = False
            if all(target == 0 for target in valid_targets[i]) or all(target == 1 for target in valid_targets[i]):
                nan = True
                info('Warning: Found a task with targets all 0s or all 1s')
            if all(pred == 0 for pred in valid_preds[i]) or all(pred == 1 for pred in valid_preds[i]):
                nan = True
                info('Warning: Found a task with predictions all 0s or all 1s')

            if nan:
                results.append(float('nan'))
                continue

        if len(valid_targets[i]) == 0:
            print(valid_targets)
            continue

        if dataset_type == 'multiclass':
            results.append(metric_func(valid_targets[i], valid_preds[i], labels=list(range(len(valid_preds[i][0])))))
        else:
            results.append(metric_func(valid_targets[i], valid_preds[i]))
            results_r2.append(r2_score(valid_targets[i], valid_preds[i]))
    return results, results_r2

In [13]:
args = TrainArgs()
args.load('args.json',skip_unsettable = True)

TrainArgs(prog='ipykernel_launcher.py', usage=None, description=':class:`TrainArgs` includes :class:`CommonArgs` along with additional arguments used for training a Chemprop model.', formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)

In [14]:
args.target_columns = ['logP']
args.target_columns

['logP']

In [15]:
args.separate_test_path = 'SGNN_dataset(test).csv'

In [16]:
if args.separate_test_path:
    test_data = get_data(path=args.separate_test_path, args=args, features_path=args.separate_test_features_path)

2053it [00:00, 127261.67it/s]
100%|██████████| 2053/2053 [00:00<00:00, 517918.09it/s]


In [17]:
test_smiles, test_targets = test_data.smiles(), test_data.targets()

In [18]:
cache = True
num_workers = 8

In [19]:
test_data_loader = MoleculeDataLoader(
    dataset=test_data,
    batch_size=args.batch_size,
    num_workers=num_workers,
    cache=cache
)

In [20]:
loss_func = get_loss_func(args)
metric_func = get_metric_func(metric=args.metric)

In [21]:
test_data1 = pd.read_csv(args.separate_test_path)

In [22]:
features_scaler = None

In [23]:
scaler = StandardScaler().fit(test_data1['logP'])

In [24]:
test_preds = predict(
            model=model,
            data_loader=test_data_loader,
            scaler=scaler,
            args=args
        )
test_scores, test_r2 = evaluate_predictions(
    preds=test_preds,
    targets=test_targets,
    num_tasks=args.num_tasks,
    metric_func=metric_func,
    dataset_type=args.dataset_type,
)


100%|██████████| 42/42 [00:09<00:00,  4.31it/s]

[[]]


In [25]:
ERROR_COLUMN = 'Absolute Error'
SMILES_COLUMN = 'smiles'
VALUE_COLUMN = 'logP'
PREDS_COLUMN = f'logP_pred{Seed}'

In [26]:
test_predictions = pd.DataFrame(columns=[SMILES_COLUMN,VALUE_COLUMN,PREDS_COLUMN])

In [27]:
test_predictions[SMILES_COLUMN] = test_smiles
test_predictions[VALUE_COLUMN] = np.array(test_targets, dtype=float).reshape(-1)

In [28]:
test_preds = [[i[0]] for i in test_preds]
test_preds

[[7.022832031252991],
 [1.5075629770730503],
 [3.812818174239756],
 [1.925955851485442],
 [2.27181981826701],
 [1.121237517471878],
 [3.896897265828306],
 [1.446388337709693],
 [5.835662398477741],
 [8.10034117381447],
 [0.5279791061783619],
 [3.5105299579875773],
 [3.363900431240164],
 [4.5808968520057185],
 [2.312286730685061],
 [5.278894422890044],
 [0.9365500060079754],
 [0.32916465708779175],
 [1.068802958367256],
 [1.3519938506230362],
 [0.5449431197839212],
 [-0.2638531687323873],
 [3.349321934905761],
 [0.971906795811796],
 [2.2460535107648707],
 [2.658463092619392],
 [2.75634478401001],
 [3.432094623907788],
 [2.384618828671243],
 [3.0855132028616774],
 [7.042779282379071],
 [0.4955153762466715],
 [1.0761345017512054],
 [1.749180353128867],
 [2.951928649576702],
 [2.754419206268931],
 [1.2276685974768982],
 [0.6158709132105993],
 [3.8492686054869347],
 [1.4638944787764203],
 [3.40164117389497],
 [-0.5679183428094645],
 [1.705167822075137],
 [5.422188829322911],
 [3.27776792643

In [29]:
test_predictions[PREDS_COLUMN] = np.array(test_preds, dtype=float).reshape(-1)

In [30]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
r2 = r2_score(test_predictions['logP'], test_preds)
rmse = np.sqrt(mean_squared_error(test_predictions['logP'], test_preds))
mae = mean_absolute_error(test_predictions['logP'], test_preds)
round(rmse, 3), round(r2, 3), round(mae, 3)

(0.453, 0.942, 0.286)

In [31]:
test_predictions.to_csv('Seeds_SGNN_PREDICT_SGNN_train_SGNN_test.csv', index=False)